In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

from collections import defaultdict, OrderedDict, Counter
import copy
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from datetime import date, datetime
import os

Populating the interactive namespace from numpy and matplotlib


# Clustering
  - klasifikace bez labelů
  - [Visualizace k-means clusterování](http://stanford.edu/class/ee103/visualizations/kmeans/kmeans.html)
  - [sklearn.clustering](https://scikit-learn.org/stable/modules/clustering.html)

# Podobné itemy

Chceme vytvořit systém, který vyhledá N podobných itemů (aut) na základě jejich parametrů.  Postup bude následující.

- vytvoření vektorů na základě kterých můžeme měřit podobnost
- vyhledání 5*N nejbližších vektorů k base vektoru
- naclusterování vybraných vektorů do N clusterů (kvůli diversifikaci)
- výběr zástupce z každého clusteru

Existuje samozřejmě více způsobů, jak danou úlohy vyřešit, ale tento způsob byl zvolen z důvodu možnosti vysvětlení následujících nástrojů:

 - vyhledávání podobných vektorů
 - clusterování
 - redukce dimenzionality pomocí tnse

In [2]:
n_recommend = 3             # number of items to recommend
n_select = 5*n_recommend+1  # number of most similar items to retrieve (will be then clustered)
base_id = 10                # id of an item for which we want to get similar items

# Vytvoření vektorů
Pipeline jako obvykle

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from train import xy_split, FeatureTransformer, FeatureScaler

In [4]:
df_train = pd.read_csv("data/data_clean_train.csv")

# Preprocess - split data to x and y, keep x as pd.DataFrame
# We need y to train the model and get feature importances - explained later
logging.info("Preprocessing")
df_x_train, y_train = xy_split(df_train)

# Features
feature_transformer = FeatureTransformer()
x_train = feature_transformer.fit_transform(df_x_train)
feature_names = feature_transformer.get_feature_names()

# Impute - fill missing values with median of the column
imputer = SimpleImputer(strategy="median")
x_train = imputer.fit_transform(x_train)

# Scale - transforms columns to be around 0
# For some methods easier training, better results, for some methods worse
# Scaling is necessary to trat each feature equally and not based on its size
features_to_scale = ["city mpg trans__", "Year__", "Number of Doors__", "Engine HP__"]
scaler = FeatureScaler(StandardScaler(), feature_names, features_to_scale)
x_train = scaler.fit_transform(x_train)

# Fit
# We need to train the model and get feature importances - explained later
logging.info("Fitting")
model = RandomForestRegressor(n_estimators=10)
model.fit(x_train, y_train)

pass  # Don't print the model

In [5]:
# Add column
df_train["cluster"] = "no_cluster"

# Change the column order
columns_order = ["cluster"] + [col for col in df_train.columns if col != "cluster"]
df_train = df_train[columns_order]

# Vážení features

Aby měly větší váhů pro podobnosti itemů důležitější features.

Použité `feature_importances_` z naučeného modelu. Váhy by se daly i naučit, pokud
bychom měli trénovací sadu ve tvaru (item, podobný item, nepodobný item).

In [6]:
x_train *= model.feature_importances_

# Vyhledání nejbližších vektorů
Mohli bychom vyhledat a rovnou doporučit `n_items_recommend` itemů. Často ale bývá problém to, že itemy jsou si moc podobné a uživatel si přeje vidět více rozdílné, diverzifikované výsledky.

In [7]:
from sklearn.neighbors import NearestNeighbors

# Class that provides nearest neighbors search
nbrs = NearestNeighbors(n_neighbors=n_select, metric='euclidean', algorithm="auto").fit(x_train)
# We find closest vectors to base item
distances, selected_ids = nbrs.kneighbors([x_train[base_id, :]])

# nbrs.kneighbors() works for 2D array (batches), I provided only one example so I take 0th result
selected_ids = selected_ids[0, :].tolist()  
# The closest item to base is also base item. I don't want to recommend it so I remove it
selected_ids.remove(base_id)

# Show closest items
# Base item is the first
df_train.loc[[base_id] + selected_ids, :]

,cluster,Unnamed: 0,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
10,no_cluster,1406,Nissan,Altima,2015,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,29830
3870,no_cluster,1408,Nissan,Altima,2015,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,26450
438,no_cluster,1417,Nissan,Altima,2016,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,27990
4492,no_cluster,1418,Nissan,Altima,2016,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,32690
3092,no_cluster,1411,Nissan,Altima,2016,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,27390
7074,no_cluster,1552,Cadillac,ATS,2016,premium unleaded (recommended),272.0,4.0,AUTOMATIC,rear wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,31,22,1624,35245
4073,no_cluster,1549,Cadillac,ATS,2016,premium unleaded (recommended),272.0,4.0,AUTOMATIC,all wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,30,22,1624,37245
358,no_cluster,1548,Cadillac,ATS,2016,premium unleaded (recommended),272.0,4.0,AUTOMATIC,rear wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,31,22,1624,43155
3845,no_cluster,1423,Nissan,Altima,2017,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,27990
3235,no_cluster,1560,Cadillac,ATS,2017,premium unleaded (recommended),272.0,4.0,AUTOMATIC,all wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,30,22,1624,40395


# Diverzifikace clusterováním
Pomocí nearest neighbors jsme dostali hodně podobná auta, ale dost často příliš podobná. Proto použijeme clusterování do tolika tříd kolik chceme doporučit itemů. A z každého clusteru pak vybereme jeden item který doporučíme.

In [8]:
from sklearn.cluster import KMeans

# Get vectors for selected items
x_selected = x_train[selected_ids, :]
# Run clustering, number of clusters is nr of items we want to recommend
# fit calculates centroids of clusters
kmeans = KMeans(n_clusters=n_recommend, random_state=0).fit(x_selected)

# Get item_ids for each cluster
clusters_item_ids = defaultdict(list)
for item_id, cluster in zip(selected_ids, kmeans.labels_):
    clusters_item_ids[cluster].append(item_id)
print("\nItem ids for clusters:")
pprint(clusters_item_ids)
    
# === Not important ===
# Store clusters to data frame
df_train["cluster"] = "no_cluster"               
df_train.loc[base_id, "cluster"] = "base_item"   
df_train.loc[selected_ids, "cluster"] = ["cluster_{}".format(cluster) for cluster in kmeans.labels_]  

# Print results
df_train.loc[[base_id] + selected_ids]


Item ids for clusters:
defaultdict(<class 'list'>,
            {0: [1900, 6051, 4842, 4905, 5819],
             1: [7074, 4073, 358, 3235, 1963],
             2: [3870, 438, 4492, 3092, 3845]})


,cluster,Unnamed: 0,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
10,base_item,1406,Nissan,Altima,2015,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,29830
3870,cluster_2,1408,Nissan,Altima,2015,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,26450
438,cluster_2,1417,Nissan,Altima,2016,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,27990
4492,cluster_2,1418,Nissan,Altima,2016,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,32690
3092,cluster_2,1411,Nissan,Altima,2016,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,27390
7074,cluster_1,1552,Cadillac,ATS,2016,premium unleaded (recommended),272.0,4.0,AUTOMATIC,rear wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,31,22,1624,35245
4073,cluster_1,1549,Cadillac,ATS,2016,premium unleaded (recommended),272.0,4.0,AUTOMATIC,all wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,30,22,1624,37245
358,cluster_1,1548,Cadillac,ATS,2016,premium unleaded (recommended),272.0,4.0,AUTOMATIC,rear wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,31,22,1624,43155
3845,cluster_2,1423,Nissan,Altima,2017,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,27990
3235,cluster_1,1560,Cadillac,ATS,2017,premium unleaded (recommended),272.0,4.0,AUTOMATIC,all wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,30,22,1624,40395


# Výběr itemů k doporučení

Z každého clusteru vybereme nyní jednoho reprezentatnta a doporučíme ho, máme tedy tolik doporučení kolik clusterů.

Kvůli jednoduchosti vybereme první item z clusteru. Lepší přístup by bylo vybrat item, který leží nejblíže centroid daného clusteru.
```
cluster_nbr = NearestNeighbor(n_neighbors=1).fit(...)
_, cluster_representative = cluster_nbr.kneighbors([kmeans.cluster_centers_[cluster, :]])
```

In [9]:
recommended_ids = [item_ids[0] for item_ids in clusters_item_ids.values()]

# Show closest examples
# Base item is the first
df_train.loc[[base_id] + recommended_ids]

,cluster,Unnamed: 0,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
10,base_item,1406,Nissan,Altima,2015,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,29830
3870,cluster_2,1408,Nissan,Altima,2015,regular unleaded,270.0,6.0,AUTOMATIC,front wheel drive,4.0,Performance,Midsize,Sedan,32,22,2009,26450
7074,cluster_1,1552,Cadillac,ATS,2016,premium unleaded (recommended),272.0,4.0,AUTOMATIC,rear wheel drive,4.0,"Luxury,Performance",Midsize,Sedan,31,22,1624,35245
1900,cluster_0,1599,Toyota,Avalon,2015,regular unleaded,268.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,31,21,2031,34140


# Vizualizace vektorů pomocí TSNE
Vektory se kterými nyní pracujeme mají dimenzi 10. Na plochu můžeme vykreslit pouze vektory o dimenzi 2. Pomocí TSNE převedeme vektory z dimenze 10 do dimenze 2. TSNE má tu vlastnost, že elementy blízko sebe ve vysoké dimenzi jsou blízko sebe i v nízké dimenzi.

Výpočet chvilku trvá...

In [10]:
from sklearn.manifold import TSNE
x_train_2d = TSNE(n_components=2).fit_transform(x_train)

print("x_train.shape={}".format(x_train.shape))
print("x_train_2d.shape={}".format(x_train_2d.shape))

x_train.shape=(7144, 10)
x_train_2d.shape=(7144, 2)


In [11]:
# Add tsne vectors to dataframe
df_train["tsne_x1"] = x_train_2d[:, 0]
df_train["tsne_x2"] = x_train_2d[:, 1]

In [12]:
# Create colors for plotting
colormap = {'no_cluster': 'gray', "base_item": 'red', 'cluster_0': 'blue', 'cluster_1': 'green', 'cluster_2': 'yellow' }
df_train["color"] = [colormap[x] for x in df_train['cluster']]

Použijeme bokeh knihovnu pro kreslení interaktivních grafů

In [13]:
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import LabelSet, ColumnDataSource, HoverTool

source = ColumnDataSource(df_train)

# HoverTool show us details when mouse on point
hover_tsne = HoverTool(tooltips=[("Make", "@Make"), ("Model", "@Model"), ("Cluster", "@cluster")])
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']

p = figure(title="Cars", tools=tools_tsne, plot_width=900, plot_height=600, match_aspect=True)
p.xaxis.axis_label = 'tsne_x1'
p.yaxis.axis_label = 'tsne_x2'

p.circle(x="tsne_x1", y="tsne_x2", fill_alpha=0.2, color='color', source=source, size=10)

output_notebook()
show(p)

Loading BokehJS ...

# Poznámky


## Použití clustrování
  - diverzifikace (viz. výše)
  - klasifikace bez labels
  - rozdělení zákazníků a použít jiný marketing pro každou skupinu
  - zřízení N výdejních míst - clustering doručovacích adres
  - vektorová kvantizace / komprese
  - použití pro diskretizaci spojitých features - [`KBinsDiscretizer(strategy='kmeans')`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html#sklearn.preprocessing.KBinsDiscretizer)

## Hledání podobných itemů
  - pokud je itemů hodně, je to náročné na výpočet. Asi nejlepší knihovna pro tento účel je [FAISS](https://github.com/facebookresearch/faiss) of Facebook.
  - používá se i na hledání podobných obrázků. Je potřeba obrázky převést na content vektory, to se dělá pomocí konvolučních  neuronových sítí.
  
## Snížení dimenzionality
  - TSNE se hodí pro vizualizace
  - Dále můžeme použít pro urychlení modelu - tam by se hodilo spíš [Principal Component analysis (PCA)](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA)